# Machine learning with TensorFlow: Softmax only

Date: August 21-22, 2018

Neural network: Softmax only

Classification of human body motion:
- walking
- sitting down
- turning right while walking
- turning left while walking

In [304]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sg
import serial
import time
import random

## Preparing data set for training CNN

In [305]:
SAMPLE_NUMS = 260  # sec
SAMPLING_RATE = 80.0  # Hz
GYRO_RESOLUTION = 250.0 / 32768.0
ACCEL_RESOLUTION = 2.0 / 32768.0

FILE0 = './20180821_walking_straight.csv'
FILE1 = './20180821_sitting.csv'
FILE2 = './20180821_turning_left.csv'
FILE3 = './20180821_turning_right.csv'

FILE0_T = './20180822_walking_straight.csv'
FILE1_T = './20180822_sitting.csv'
FILE2_T = './20180822_turning_left.csv'
FILE3_T = './20180822_turning_right.csv'

TIME_INTERVAL = 260.0 / SAMPLING_RATE  # sec

MEASUREMENTS = 8
RECORDS = 260

In [306]:
gres = lambda v: v * GYRO_RESOLUTION
ares = lambda v: v * ACCEL_RESOLUTION
to_time = lambda v: v / SAMPLING_RATE

def conv(df):
    df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(gres)
    df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(ares)
    CUTOFF = 10.0
    b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
    df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(lambda row: sg.lfilter(b, a, row))
    CUTOFF = 10.0
    b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
    df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(lambda row: sg.lfilter(b, a, row))
    df[['cnt']] = df[['cnt']].apply(to_time)
    df.set_index('cnt', drop=True, inplace=True)
    # measurements = df.tail(1).iloc[0,0] + 1
    # return measurements

In [307]:
df0,df0_t = pd.read_csv(FILE0, dtype=np.int16),pd.read_csv(FILE0_T, dtype=np.int16)
df1,df1_t = pd.read_csv(FILE1, dtype=np.int16),pd.read_csv(FILE1_T, dtype=np.int16)
df2,df2_t = pd.read_csv(FILE2, dtype=np.int16),pd.read_csv(FILE2_T, dtype=np.int16)
df3,df3_t = pd.read_csv(FILE3, dtype=np.int16),pd.read_csv(FILE3_T, dtype=np.int16)

conv(df0), conv(df0_t)
conv(df1), conv(df1_t)
conv(df2), conv(df2_t)
conv(df3), conv(df3_t)

(None, None)

In [308]:
df_set, df0set, df1set, df2set, df3set = [], [], [], [], []
df_t_set, df0tset, df1tset, df2tset, df3tset = [], [], [], [], []
#df_set, df0set, df1set, df2set = [], [], [], []
#df_set, df0set, df1set = [], [], []
for i in range(MEASUREMENTS):
    df0set.append([df0[df0['id'] == i], [1,0,0,0]])
    df1set.append([df1[df1['id'] == i], [0,1,0,0]])
    df2set.append([df2[df2['id'] == i], [0,0,1,0]])
    df3set.append([df3[df3['id'] == i], [0,0,0,1]])
    df0tset.append([df0_t[df0_t['id'] == i], [1,0,0,0]])
    df1tset.append([df1_t[df1_t['id'] == i], [0,1,0,0]])
    df2tset.append([df2_t[df2_t['id'] == i], [0,0,1,0]])
    df3tset.append([df3_t[df3_t['id'] == i], [0,0,0,1]])

df_set.extend(df0set)
df_set.extend(df1set)
df_set.extend(df2set)
df_set.extend(df3set)

df_t_set.extend(df0tset)
df_t_set.extend(df1tset)
df_t_set.extend(df2tset)
df_t_set.extend(df3tset)

random.shuffle(df_set)
random.shuffle(df_t_set)

In [309]:
# use accel x axis and gyro z axis values for classification of human body motion
train_x, train_t_x = [], []
train_t, train_t_t = [], []
for df, label in df_set:
    values = np.concatenate((df['ax'].values, df['ay'].values, df['az'].values,
                             df['gx'].values, df['gy'].values, df['gz'].values),
                            axis=None)
    train_x.append(values)
    train_t.append(label)
for df, label in df_t_set:
    values = np.concatenate((df['ax'].values, df['ay'].values, df['az'].values,
                             df['gx'].values, df['gy'].values, df['gz'].values),
                            axis=None)
    train_t_x.append(values)
    train_t_t.append(label)

#train_x

## TensorFlow

In [310]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20180822)
tf.set_random_seed(20180822)

In [311]:
num_samples = SAMPLE_NUMS * 6
num_units = 64
num_classify = 4

x = tf.placeholder(tf.float32, [None, num_samples])

w1 = tf.Variable(tf.truncated_normal([num_samples, num_units]))
b1 = tf.Variable(tf.zeros([num_units]))
hidden1 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)

w0 = tf.Variable(tf.zeros([num_units, num_classify]))
b0 = tf.Variable(tf.zeros([num_classify]))
p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0)

In [312]:
t = tf.placeholder(tf.float32, [None, num_classify])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer().minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [313]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [314]:
i = 0
for _ in range(2000):
    i += 1
    sess.run(train_step, feed_dict={x:train_x, t:train_t})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:train_t_x, t: train_t_t})
        print('Step: {}, Loss: {}, Accuracy: {}'.format(i, loss_val, acc_val))

Step: 100, Loss: 19.662357330322266, Accuracy: 0.875
Step: 200, Loss: 13.925043106079102, Accuracy: 0.875
Step: 300, Loss: 12.304342269897461, Accuracy: 0.875
Step: 400, Loss: 11.863088607788086, Accuracy: 0.875
Step: 500, Loss: 11.458113670349121, Accuracy: 0.875
Step: 600, Loss: 11.293909072875977, Accuracy: 0.875
Step: 700, Loss: 11.288328170776367, Accuracy: 0.875
Step: 800, Loss: 11.287398338317871, Accuracy: 0.875
Step: 900, Loss: 11.304747581481934, Accuracy: 0.875
Step: 1000, Loss: 11.334665298461914, Accuracy: 0.875
Step: 1100, Loss: 11.394475936889648, Accuracy: 0.875
Step: 1200, Loss: 11.459219932556152, Accuracy: 0.875
Step: 1300, Loss: 11.523523330688477, Accuracy: 0.875
Step: 1400, Loss: 11.58803653717041, Accuracy: 0.875
Step: 1500, Loss: 11.635093688964844, Accuracy: 0.875
Step: 1600, Loss: 11.320343017578125, Accuracy: 0.875
Step: 1700, Loss: 11.359342575073242, Accuracy: 0.875
Step: 1800, Loss: 11.3921480178833, Accuracy: 0.875
Step: 1900, Loss: 11.432107925415039, Ac

In [315]:
p_test = sess.run(p, feed_dict={x:train_t_x})
(p_test*100).astype(int)[:10]

array([[98,  0,  0,  1],
       [ 0, 97,  0,  1],
       [ 0,  0, 99,  0],
       [10,  5, 81,  1],
       [ 0,  0, 99,  0],
       [ 9,  1, 89,  0],
       [ 1,  0,  0, 98],
       [ 0,  0, 99,  0],
       [ 0, 99,  0,  0],
       [ 1,  0,  0, 98]])

In [316]:
train_t_t[:10]

[[1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1]]

In [317]:
sess.close()